In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from matplotlib import style 
import seaborn as sns 
%matplotlib inline
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

In [2]:
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.naive_bayes import GaussianNB 
import xgboost as xgb 
import lightgbm as lgbm 
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.model_selection import train_test_split,cross_validate 
from sklearn.model_selection import KFold 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import cross_val_score

In [ ]:
df = pd.read_csv(r'Provide the directory path for the "Healthcare_Dataset_Stroke_Data.csv" here, such as "C:/Project/Dataset/Healthcare_Dataset_Stroke_Data.csv"')
df. head ()

In [ ]:
df. shape

In [ ]:
df.info()

In [ ]:
fig, axes = plt.subplots(nrows = 5, ncols = 2, figsize = (12, 20))
sns.boxplot(x = 'avg_glucose_level', data = df, ax=axes [0][0])
sns.boxplot(x = 'age', data = df, ax=axes [1][0])
sns.boxplot(x = 'bmi', data = df, ax=axes [2][0])
sns.boxplot(x = 'smoking_status',y = 'age', hue = 'stroke',data = df, ax=axes[3][0])
sns.boxplot(x = 'hypertension', y ='age', hue = 'stroke', data = df, ax=axes[3] [1])
sns.boxplot(x = 'heart_disease',y = 'age', hue = 'stroke', data = df, ax=axes [0][1])
sns.boxplot(x = 'ever_married',y = 'age', hue = 'stroke', data = df, ax=axes [1][1])
sns.boxplot(x = 'work_type',y = 'age',hue = 'stroke', data = df, ax=axes[2][1])
sns.boxplot(x = 'Residence_type', y ='age', hue = 'stroke',data = df, ax = axes[4][0]) 
sns.boxplot(x = 'smoking_status', y = 'age', hue = 'stroke',data = df,ax = axes [4][1])

In [ ]:
sns.barplot(x = 'stroke' ,y='smoking_status', data = df)

In [ ]:
replace_values = {'Unknown': 'never smoked'}
df = df.replace({'smoking_status' : replace_values})
df.head()

In [ ]:
df.describe()

In [ ]:
df=df.drop('id',axis=1)
df.shape

In [ ]:
df.isnull().sum()

In [12]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

In [14]:
def remove_outliers(data):
    arr=[]
    q1=np.percentile(data, 25) 
    q3=np.percentile(data, 75) 
    iqr=q3-q1 
    mi=q1-(1.5*iqr) 
    ma=q3+(1.5*iqr)
    for i in list(data):
        if i<mi:
            i=mi 
            arr.append(i)
        elif i>ma:
            i=ma 
            arr.append(i)
        else:
            arr.append(i)
    return arr

In [ ]:
df['bmi'] = remove_outliers (df['bmi'])
df['avg_glucose_level'] = remove_outliers(df['avg_glucose_level'])
print('Outliers successfully removed')

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (10, 5))
sns.boxplot(x = 'bmi', data = df, ax=axes [1][0])
sns.boxplot(x = 'avg_glucose_level', data = df, ax=axes [0][0])

In [ ]:
for i in ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']:
    print(df[i].unique())

In [18]:
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
le2 = LabelEncoder()
df['Residence_type'] = le1.fit_transform(df['Residence_type'])
df['ever_married'] = le2.fit_transform(df['ever_married'])

In [ ]:
import joblib
joblib.dump(le1,"res_transform")
joblib.dump(le2,"mar_transform")

In [ ]:
df.shape

In [ ]:
df.iloc[0,:]

In [ ]:
df.head()

In [23]:
x = df.iloc[:,0:10].values
y = df.iloc[:,10].values

In [24]:
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder
ct= ColumnTransformer([("onehot", OneHotEncoder (drop= 'first'), [0,5,9])], remainder= "passthrough")
x=ct.fit_transform(x)

In [ ]:
joblib.dump(ct,"column")

In [ ]:
x

In [ ]:
x[0]

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
sns.countplot(y)

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
x_res, y_res = sm.fit_resample(x,y)
print("Before OverSampling, counts of label '1': {}".format(sum(y==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum (y==0)))

print('After OverSampling, the shape of train_X: {}'.format(x_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format (y_res.shape))

print ("After OverSampling, counts of label '1': {}".format(sum(y_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_res==0)))

In [ ]:
sns.countplot(y_res)

In [33]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size = 0.2, random_state = 42)

In [ ]:
dtc = DecisionTreeClassifier()
dtc.fit(x_train,y_train)
dtc_pred = dtc.predict(x_test)
print('***Decision Tree Model Results***')
print(confusion_matrix(dtc_pred, y_test))
print(classification_report(dtc_pred, y_test))

In [ ]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf_pred = rf.predict(x_test)
print('***Random Forest***')
print(confusion_matrix(rf_pred, y_test))
print(classification_report(rf_pred, y_test))

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
lr_pred = lr.predict(x_test)
print(confusion_matrix(lr_pred, y_test))
print('***Logistic Regression***')
print (classification_report(lr_pred, y_test))

In [ ]:
svc = SVC()
svc.fit(x_train, y_train)
svc_pred = svc.predict(x_test)
print('***Support vector classifier***')
print (confusion_matrix(svc_pred, y_test))
print(classification_report(svc_pred, y_test))

In [ ]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
knn_pred = knn.predict(x_test)
print('***K-Nearest Neighbours***')
print(confusion_matrix(knn_pred, y_test))
print(classification_report(knn_pred, y_test))

In [39]:
cross_valid_scores={}

In [ ]:
%%time
parameters = {
"max_depth": [3, 5, 7, 9, 11, 13],
}
model_dtc = DecisionTreeClassifier(
random_state=42,
class_weight='balanced',
)
model_dtc = GridSearchCV(
model_dtc, 
parameters, 
cv=5,
)
model_dtc.fit(x_train, y_train)
model_dtc_pred = model_dtc.predict(x_test)
print('*** Applying Grid Search to Decision tree***')
print(classification_report(model_dtc_pred, y_test))
print(f'Best parameters {model_dtc.best_params_}')
print(
f'Mean cross-validated accuracy score of the best estimator: ' + \
f'{model_dtc.best_score_:.3f}'
)
cross_valid_scores['desicion_tree'] = model_dtc.best_score_
print('-----')

In [ ]:
%%time
parameters = {
    "n_estimators": [5, 10, 15, 20, 25],
    "max_depth": [3, 5, 7, 9, 11, 13],
}

model_rf = RandomForestClassifier(
    random_state=42,
    class_weight='balanced',
)

model_rf = GridSearchCV(
    model_rf,
    parameters,
    cv=5,
)

model_rf.fit(x_train, y_train)
model_rf_pred = model_rf.predict(x_test)

print('*** Applying Grid Search to Random Forest ***')
print(classification_report(model_rf_pred, y_test))
print(f'Best parameters: {model_rf.best_params_}')
print(f'Mean cross-validated accuracy score of the best_estimator: '+ \
      f'{model_rf.best_score_:.3f}')
cross_valid_scores['random_forest'] = model_rf.best_score_
print('-----')


In [ ]:
import joblib
joblib.dump(model_rf,"model")